### Load data

In [2]:
import numpy as np
X_test = np.load("X_test.npy")
y_test = np.load("y_test.npy")
person_train_valid = np.load("person_train_valid.npy")
X_train_valid = np.load("X_train_valid.npy")
y_train_valid = np.load("y_train_valid.npy")
person_test = np.load("person_test.npy")
print(np.unique(y_train_valid))
print(np.unique(y_test))

[769 770 771 772]
[769 770 771 772]


### Shape of data

In [3]:
print ('Training/Valid data shape: {}'.format(X_train_valid.shape))
print ('Test data shape: {}'.format(X_test.shape))
print ('Training/Valid target shape: {}'.format(y_train_valid.shape))
print ('Test target shape: {}'.format(y_test.shape))
print ('Person train/valid shape: {}'.format(person_train_valid.shape))
print ('Person test shape: {}'.format(person_test.shape))

Training/Valid data shape: (2115, 22, 1000)
Test data shape: (443, 22, 1000)
Training/Valid target shape: (2115,)
Test target shape: (443,)
Person train/valid shape: (2115, 1)
Person test shape: (443, 1)


### Prepare Training Data

In [4]:
import torch

train_on_gpu = torch.cuda.is_available()

# check if CUDA is available
device = torch.device("cuda:0" if train_on_gpu else "cpu")
print(device)
if not train_on_gpu:
    print('Training on CPU ...')
else:
    print('Training on GPU ...')

cuda:0
Training on GPU ...


In [5]:
from torch.utils import data

N, H, W = X_train_valid.shape
X_train_valid = torch.from_numpy(X_train_valid.reshape(N,1,H,W)).float().to(device)
y_train_valid = torch.from_numpy(y_train_valid).long().to(device)
train_data = data.TensorDataset(X_train_valid, y_train_valid)

person_train_valid = torch.from_numpy(person_train_valid).to(device)
person_test = torch.from_numpy(person_test).to(device)

In [6]:
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler

batch_size = 200
valid_size = 0.2

"""
# convert data to a normalized torch.FloatTensor
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])
"""

# obtain training indices that will be used for validation
num_train = len(train_data)
indices = list(range(num_train))
np.random.shuffle(indices)
split = int(np.floor(valid_size * num_train))
train_idx, valid_idx = indices[split:], indices[:split]

# define samplers for obtaining training and validation batches
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

# prepare data loaders (combine dataset and sampler)
train_loader = data.DataLoader(train_data, batch_size=batch_size, sampler=train_sampler)
valid_loader = data.DataLoader(train_data, batch_size=batch_size, sampler=valid_sampler)

## CNN Architecture

### AlexNet (modified)

In [12]:
import torch.nn as nn
import torch.nn.functional as F

class AlexNet(nn.Module):

    def __init__(self, num_classes=1000):
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=(4,12), stride=4, padding=(0,2)),
            # want to capture relation in both dims, pad only time dim
            #nn.BatchNorm2d()
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=(2,3), stride=(1,2)),
            nn.Conv2d(64, 192, kernel_size=(3,5), padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=(1,2)),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        self.classifier = nn.Sequential(
            nn.Dropout(p=0.2),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.2),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

# create a complete CNN
model = AlexNet()

# move tensors to GPU if CUDA is available
if train_on_gpu:
    model.cuda()

### Train the Model

In [13]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), weight_decay=0.0002, lr=5e-3)

# number of epochs to train the model
n_epochs = 30

valid_loss_min = np.Inf # track change in validation loss

for epoch in range(1, n_epochs+1):

    # keep track of training and validation loss
    train_loss = 0.0
    valid_loss = 0.0
    
    ###################
    # train the model #
    ###################
    model.train()
    for data, target in train_loader:
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()*data.size(0)
        
    ######################    
    # validate the model #
    ######################
    model.eval()
    for data, target in valid_loader:
        output = model(data)
        loss = criterion(output, target)
        valid_loss += loss.item()*data.size(0)
    
    # calculate average losses
    train_loss = train_loss/len(train_loader.sampler)
    valid_loss = valid_loss/len(valid_loader.sampler)
        
    # print training/validation statistics 
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
        epoch, train_loss, valid_loss))
    
    # save model if validation loss has decreased
    if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
        valid_loss_min,
        valid_loss))
        torch.save(model.state_dict(), 'model_cifar.pt')
        valid_loss_min = valid_loss

Epoch: 1 	Training Loss: 23193.086772 	Validation Loss: 5.528068
Validation loss decreased (inf --> 5.528068).  Saving model ...
Epoch: 2 	Training Loss: 3.508127 	Validation Loss: 2.449748
Validation loss decreased (5.528068 --> 2.449748).  Saving model ...


PermissionError: [Errno 13] Permission denied: 'model_cifar.pt'

### Load the Model with the Lowest Validation Loss

In [10]:
model.load_state_dict(torch.load('model_cifar.pt'))

<All keys matched successfully>

### Test the Trained Network

In [11]:
from torch.utils import data

batch_size = 200

N, H, W = X_test.shape
X_test = torch.from_numpy(X_test.reshape(N,1,H,W)).float().to(device)
y_test = torch.from_numpy(y_test).long().to(device)
test_data = data.TensorDataset(X_test, y_test)

test_loader = data.DataLoader(test_data, batch_size=batch_size)

In [12]:
# specify the image classes
classes = [769, 770, 771, 772]

# track test loss
test_loss = 0.0
class_correct = list(0. for i in range(4))
class_total = list(0. for i in range(4))

model.eval()
for data, target in test_loader:
    output = model(data)
    loss = criterion(output, target)
    # update test loss 
    test_loss += loss.item()*data.size(0)
    # convert output probabilities to predicted class
    _, pred = torch.max(output, 1)
    # compare predictions to true label
    correct_tensor = pred.eq(target.data.view_as(pred))
    correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
    print(correct.shape)
    # calculate test accuracy for each object class
    for i in range(correct.shape[0]):
        label = target.data[i]
        class_correct[label-769] += correct[i].item()
        class_total[label-769] += 1
# average test loss
test_loss = test_loss/len(test_loader.dataset)
print('Test Loss: {:.6f}\n'.format(test_loss))

for i in range(4):
    if class_total[i] > 0:
        print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
            classes[i], 100 * class_correct[i] / class_total[i],
            np.sum(class_correct[i]), np.sum(class_total[i])))
    else:
        print('Test Accuracy of %5s: N/A (no training examples)' % (classes[i]))

print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (
    100. * np.sum(class_correct) / np.sum(class_total),
    np.sum(class_correct), np.sum(class_total)))

(200,)
(200,)
(43,)
Test Loss: 1.391818

Test Accuracy of   769:  0% ( 0/111)
Test Accuracy of   770:  0% ( 0/127)
Test Accuracy of   771: 100% (96/96)
Test Accuracy of   772:  0% ( 0/109)

Test Accuracy (Overall): 21% (96/443)


In [ ]:
class EEGDataset(data.dataset):
    """Custom PyTorch dataset for EEG dataset"""

    def __init__(self, np_darray, transform=None):
        """
        Args:
            np_darray: np array containing the EEG dataset
            transform: optional transform to be applied on a sample
        """
        self.

In [ ]:


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1) # conv layer (sees 32x32x3 tensor)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1) # conv layer (sees 16x16x16 tensor)
        self.conv3 = nn.Conv2d(32, 64, 3, padding=1) # conv layer (sees 8x8x32 tensor)
        self.pool = nn.MaxPool2d(2, 2) # max pooling layer
        self.fc1 = nn.Linear(64 * 4 * 4, 500) # linear layer (64 * 4 * 4 -> 500)
        self.fc2 = nn.Linear(500, 10) # linear layer (500 -> 10)
        self.dropout = nn.Dropout(0.25) # dropout layer (p=0.25)

    def forward(self, x):
        # add sequence of convolutional and max pooling layers
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        # flatten image input
        x = x.view(-1, 64 * 4 * 4)
        # add dropout layer
        x = self.dropout(x)
        # add 1st hidden layer, with relu activation function
        x = F.relu(self.fc1(x))
        # add dropout layer
        x = self.dropout(x)
        # add 2nd hidden layer, with relu activation function
        x = self.fc2(x)
        return x

# create a complete CNN
model = Net()
print(model)

# move tensors to GPU if CUDA is available
if train_on_gpu:
    model.cuda()